https://towardsdatascience.com/probability-concepts-explained-maximum-likelihood-estimation-c7b4342fdbb1

# Введение
В этой статье я объясню, что такое метод максимального правдоподобия для оценки параметров, и приведу простой пример, демонстрирующий этот метод. Часть материала требует знания фундаментальных концепций вероятности, таких как определение совместной вероятности и независимости событий. [Я написал пост в блоге с этими предварительными условиями](https://medium.com/@jonnybrooks04/probability-concepts-explained-introduction-a7c0316de465), так что не стесняйтесь читать его, если считаете, что вам нужно освежить знания.

# Что такое параметры?
Часто в машинном обучении мы используем модель для описания процесса, который приводит к наблюдаемым данным. Например, мы можем использовать модель случайного леса для классификации того, могут ли клиенты отменить подписку на услугу (известную как моделирование оттока), или мы можем использовать линейную модель для прогнозирования дохода, который будет получен для компании в зависимости от того, сколько они могут потратить на рекламу (это будет примером линейной регрессии). Каждая модель содержит свой собственный набор параметров, который в конечном итоге определяет, как выглядит эта модель.

Для линейной модели мы можем записать это как $y = mx + c$. В этом примере $x$ может представлять расходы на рекламу, а $y$ может быть полученным доходом. $m$ и $c$ являются параметрами для этой модели. Различные значения этих параметров дадут разные линии (см. рисунок ниже).

<img src='https://miro.medium.com/v2/resize:fit:1400/format:webp/1*YE0OsCA9xug9fndqk7YGkg.png' width=500>

*Три линейные модели с различными значениями параметров.*

Итак, параметры определяют схему модели. Только когда для параметров выбираются конкретные значения, мы получаем реализацию модели, описывающей данное явление.

# Интуитивное объяснение оценки максимального правдоподобия
Оценка максимального правдоподобия — это метод, который определяет значения параметров модели. Значения параметров находятся таким образом, чтобы они максимизировали вероятность того, что процесс, описанный моделью, произвел данные, которые фактически наблюдались.

Приведенное выше определение все еще может звучать немного загадочно, поэтому давайте рассмотрим пример, чтобы помочь понять это.

Предположим, что мы наблюдали 10 точек данных из некоторого процесса. Например, каждая точка данных может представлять собой время в секундах, которое требуется студенту, чтобы ответить на определенный вопрос экзамена. Эти 10 точек данных показаны на рисунке ниже.

<img src='https://miro.medium.com/v2/resize:fit:1100/format:webp/1*Z3JJGvEtOjmpLFvmWiUR3Q.png' width=500>

*10 (гипотетических) точек данных, которые мы наблюдали*

Сначала нам нужно решить, какая модель, по нашему мнению, лучше всего описывает процесс генерации данных. Эта часть очень важна. По крайней мере, мы должны иметь хорошее представление о том, какую модель использовать. Обычно это достигается с помощью некоторого опыта в предметной области, но мы не будем обсуждать это здесь.

Для этих данных мы предположим, что процесс генерации данных может быть адекватно описан гауссовым (нормальным) распределением. Визуальный осмотр рисунка выше показывает, что гауссово распределение правдоподобно, поскольку большинство из 10 точек сгруппированы в середине, а несколько точек разбросаны слева и справа. (Принимать такое решение на лету, имея всего 10 точек данных, неразумно, но, учитывая, что я сгенерировал эти точки данных, мы будем придерживаться этого).

Напомню, что гауссово распределение имеет 2 параметра. Среднее значение, $μ$, и стандартное отклонение, $σ$. Различные значения этих параметров приводят к различным кривым (как и в случае с прямыми линиями выше). Мы хотим узнать, какая кривая, скорее всего, ответственна за создание точек данных, которые мы наблюдали? (См. рисунок ниже). Оценка максимального правдоподобия — это метод, который находит значения $μ$ и $σ$, которые приводят к кривой, наилучшим образом соответствующей данным.

<img src='https://miro.medium.com/v2/resize:fit:1400/format:webp/1*uLKl0Nz1vFg6bmfiqpCKZQ.png' width=500>

*10 точек данных и возможные гауссовские распределения, из которых были взяты данные. $f1$ нормально распределена со средним значением 10 и дисперсией 2,25 (дисперсия равна квадрату стандартного отклонения), это также обозначается $f_1 ~ N (10, 2.25)$, $f_2 ~ N (10, 9)$, $f_3 ~ N (10, 0.25)$ и $f_4 ~ N (8, 2.25)$. Цель максимального правдоподобия — найти значения параметров, которые дают распределение, максимизирующее вероятность наблюдения данных.*

Истинное распределение, на основе которого были получены данные, было $f1 ~ N$(10, 2,25), что соответствует синей кривой на рисунке выше.

# Расчет оценок максимального правдоподобия
Теперь, когда у нас есть интуитивное понимание того, что такое оценка максимального правдоподобия, мы можем перейти к изучению того, как вычислять значения параметров. Значения, которые мы находим, называются оценками максимального правдоподобия (MLE).

Мы снова продемонстрируем это на примере. Предположим, что на этот раз у нас есть три точки данных, и мы предполагаем, что они были сгенерированы из процесса, который адекватно описывается распределением Гаусса. Это точки 9, 9.5 и 11. *Как нам вычислить оценки максимального правдоподобия значений параметров распределения Гаусса $μ$ и $σ$?*

Мы хотим вычислить общую вероятность наблюдения всех данных, т. е. совместное распределение вероятностей всех наблюдаемых точек данных. Для этого нам нужно будет вычислить некоторые условные вероятности, что может оказаться очень сложным. Поэтому здесь мы сделаем наше первое предположение. *Предполагается, что каждая точка данных генерируется независимо от других*. Это предположение значительно упрощает математику. Если события (т. е. процесс, который генерирует данные) независимы, то общая вероятность наблюдения всех данных является произведением наблюдения каждой точки данных по отдельности (т. е. произведением предельных вероятностей).

Плотность вероятности наблюдения одной точки данных $x$, которая генерируется из гауссовского распределения, определяется как:

<img src='https://miro.medium.com/v2/resize:fit:1400/format:webp/1*t4zrihvhtlZJZsvcX3jRjg.png' width=500>

Точка с запятой, используемая в обозначении $P(x; μ, σ)$, подчеркивает, что символы, которые появляются после нее, являются параметрами распределения вероятностей. Поэтому ее не следует путать с условной вероятностью (которая обычно представлена ​​вертикальной линией, например $P(A| B)$).

В нашем примере общая (совместная) плотность вероятности наблюдения трех точек данных определяется как:

<img src='https://miro.medium.com/v2/resize:fit:1400/format:webp/1*rFzbQ614IR4zEwBM3k1V0Q.png' width=500>

Нам просто нужно вычислить значения $μ$ и $σ$, которые в результате дадут максимальное значение приведенного выше выражения.

Если вы изучали исчисление на уроках математики, то вы, вероятно, знаете, что существует метод, который может помочь нам найти максимумы (и минимумы) функций. Он называется *дифференциацией*. Все, что нам нужно сделать, это найти производную функции, установить производную функцию равной нулю, а затем перестроить уравнение так, чтобы интересующий параметр стал объектом уравнения. И вуаля, у нас будут наши значения MLE для наших параметров. Сейчас я пройду эти шаги, но предполагаю, что читатель знает, как выполнять дифференциацию для общих функций. Если вы хотите более подробное объяснение, просто дайте мне знать в комментариях.

# Логарифмическое правдоподобие
Вышеуказанное выражение для общей вероятности на самом деле довольно сложно дифференцировать, поэтому его почти всегда упрощают, взяв натуральный логарифм выражения. Это абсолютно нормально, поскольку натуральный логарифм является монотонно возрастающей функцией. Это означает, что если значение на оси x увеличивается, значение на оси y также увеличивается (см. рисунок ниже). Это важно, поскольку это гарантирует, что максимальное значение логарифма вероятности возникает в той же точке, что и исходная функция вероятности. Поэтому мы можем работать с более простым логарифмическим правдоподобием вместо исходного правдоподобия.

<img src='https://miro.medium.com/v2/resize:fit:1400/format:webp/1*hgz4ePKHyMh72hVrEguoyw.png' width=500>

*Монотонное поведение исходной функции $y = x$ слева и (натуральной) логарифмической функции $y = ln(x)$. Обе эти функции монотонны, поскольку при движении слева направо по оси $x$ значение $y$ всегда увеличивается.*

<img src='https://miro.medium.com/v2/resize:fit:1400/format:webp/1*EN94xeYTJgnhDFnMsHf2WA.png' width=500>

*Пример немонотонной функции, поскольку при движении слева направо по графику значение $f(x)$ увеличивается, затем уменьшается, а затем снова увеличивается.*

Логарифмирование исходного выражения дает нам:

<img src='https://miro.medium.com/v2/resize:fit:1400/format:webp/1*iEdEaqWWiruaw_Fr2ophxw.png' width=500>

Это выражение можно снова упростить, используя законы логарифмов, и получить:

<img src='https://miro.medium.com/v2/resize:fit:1400/format:webp/1*kdjQQo5jUX9a2Z0kblJ4Hg.png' width=500>

Наконец, приравнивая левую часть уравнения к нулю и затем переставляя для $μ$, получаем:

<img src='https://miro.medium.com/v2/resize:fit:1400/format:webp/1*rHtqdjFXRw4sdnLU9n_WsQ.png' width=500>

И вот у нас есть оценка максимального правдоподобия для $μ$. Мы можем сделать то же самое и с $σ$, но я оставлю это в качестве упражнения для внимательного читателя.

# Заключительные замечания


## Всегда ли можно точно решить оценку максимального правдоподобия?
Нет — вот короткий ответ. Более вероятно, что в реальном сценарии производная функции логарифмического правдоподобия все еще аналитически неразрешима (т. е. слишком сложно/невозможно дифференцировать функцию вручную). Поэтому для поиска численных решений для оценок параметров используются итерационные методы, такие как [алгоритмы максимизации ожидания](https://en.wikipedia.org/wiki/Expectation%E2%80%93maximization_algorithm). Однако общая идея остается прежней.


## Так почему же максимальное правдоподобие, а не максимальная вероятность?
Ну, это просто педантизм статистиков (но на то есть веские причины). Большинство людей склонны использовать вероятность и правдоподобие как взаимозаменяемые, но статистики и теоретики вероятностей различают эти два понятия. Причину путаницы лучше всего прояснить, взглянув на уравнение.

<img src='https://miro.medium.com/v2/resize:fit:1400/format:webp/1*XUHA8X_WauSB8anrb6lllA.png' width=500>

Эти выражения равны! Так что же это значит? Давайте сначала дадим определение $P(data; μ, σ)$? Это означает *«плотность вероятности наблюдения данных с параметрами модели $μ$ и $σ$»*. Стоит отметить, что мы можем обобщить это на любое количество параметров и любое распределение.

С другой стороны, $L(μ, σ; data)$ означает *«вероятность того, что параметры $μ$ и $σ$ примут определенные значения, учитывая, что мы наблюдали кучу данных»*.

Уравнение выше говорит, что плотность вероятности данных с учетом параметров равна правдоподобию параметров с учетом данных. Но несмотря на то, что эти две вещи равны, правдоподобие и плотность вероятности принципиально задают разные вопросы — один спрашивает о данных, а другой спрашивает о значениях параметров. Вот почему метод называется максимальным правдоподобием, а не максимальной вероятностью.

## Когда минимизация наименьших квадратов совпадает с оценкой максимального правдоподобия?
Минимизация наименьших квадратов — еще один распространенный метод оценки значений параметров для модели в машинном обучении. Оказывается, когда модель предполагается гауссовой, как в примерах выше, оценки MLE эквивалентны методу наименьших квадратов. Для более глубокого математического вывода ознакомьтесь с [этими слайдами](https://web.archive.org/web/20111202153913/http://www.cs.cmu.edu/~epxing/Class/10701/recitation/recitation3.pdf).

Интуитивно мы можем интерпретировать связь между двумя методами, понимая их цели. Для оценки параметров наименьших квадратов мы хотим найти линию, которая минимизирует общий квадрат расстояния между точками данных и линией регрессии (см. рисунок ниже). При оценке максимального правдоподобия мы хотим максимизировать общую вероятность данных. Когда предполагается гауссовское распределение, максимальная вероятность находится, когда точки данных приближаются к среднему значению. Поскольку гауссово распределение симметрично, это эквивалентно минимизации расстояния между точками данных и средним значением.

<img src='https://miro.medium.com/v2/resize:fit:1400/format:webp/1*dgdJ47sNQHxZ5t1uWutflA.png' width=500>

Линия регрессии, показывающая точки данных со случайным гауссовым *шумом*

В следующем посте я планирую рассказать о [байесовском выводе и о том, как его можно использовать для оценки параметров](https://towardsdatascience.com/probability-concepts-explained-bayesian-inference-for-parameter-estimation-90e8930e5348).